# Ejercicio 13

## Enunciado
Crea un programa que:

1. Acceda al [siguiente enlace](https://misssushi.es/localiza-tu-misssushi-mas-cercano/) y obtenga para cada restaurante, su nombre, su dirección, su email y su teléfono.
2. Guarde los datos en un fichero de texto con extensión .csv separados por ; para poder visualizarlos en Excel.

### ¿Qué cosas nuevas necesitamos saber?
- Expresiones regulares. La librería **re**.
- Leer y guardar ficheros.

### Expresiones regulares. La librería **re**.

Las expresiones regulares son cadenas de texto que conforman patrones de búsqueda. Es decir, mediante un patrón común, podemos encontrar todas aquellas cadenas de texto concuerdan con él.

Para ello, hacemos uso de la librería **re**.

Veamos un ejemplo sencillo de como utilizarlos:

In [ ]:
patron_texto = r'CARGO DE (.*) EN SU CUENTA BANCARIA' # precediento nuestro str de una r indicamos que es una expresión regular, y por lo tanto
# que se ha de respestar tal y como es. Ahora os explico que quiero decir...

En el patrón anterior, tenemos 3 elementos que utilizamos en las expresiones regulares o regex:
- Los paréntesis "()": Sirven para determinar grupos y poder recuperarlos a posteriori. El número del grupo corresponde al orden que ocupa respecto a los demás en la regex. En este caso, dado que solo hay 1, sería el primero.
- El punto ".": Sirve para indicar que ahí puede aparecer cualquier caracter.
- El asterisco "\*": Sirve para cuantificar el número de veces que aparece algo (en este caso, como está precedido por un punto, cualquier cosa) e indica que puede aparecer de 0 a infinitas veces.

Veamos un ejemplo de como usar nuestra regex:

In [ ]:
# supongamos un extracto bancario con el siguiente aspecto
extracto = [
"CARGO DE SPOTIFY EN SU CUENTA BANCARIA",
"CARGO DE NETFLIX EN SU CUENTA BANCARIA",
"CARGO DE AMAZON EN SU CUENTA BANCARIA",
"COMPRA EN MERCADONA", # ESTA LA VA A IGNORAR
"COMPRA EN ALCAMPO", # ESTA LO MISMO
"CARGO DE MAPFRE EN SU CUENTA BANCARIA"
]

In [ ]:
import re # importamos nuestra libreia

In [ ]:
patron_compilado = re.compile(patron_texto) # con esto generamos un objeto con el que poder buscar las coincidencias

In [ ]:
# y así, buscamos que cadenas de texto se corresponden con nuestro patrón
for apunte in extracto:
    # esto devuelve un Match si coincide o None en caso contrario
    coincide = patron_compilado.fullmatch(apunte)
    if coincide is not None:
        print(coincide.string)

In [ ]:
# de manera similar, podemos obtener solo grupos que queramos. En nuestro caso, saber quien nos ha realizado el cargo
for apunte in extracto:
    # esto devuelve un Match si coincide o None en caso contrario
    coincide = patron_compilado.fullmatch(apunte)
    if coincide: # asi también se comprueba que algo no es None
        print(coincide.group(1))

Podéis practicar y aprender más sobre expresiones regulares en varias páginas de internet.

A mí, personalmente, me gusta esta: [Regex101](https://regex101.com/).

### Leer y guardar ficheros.

Para leer y escribir ficheros, es tan simple como lo siguiente:

#### Para Leer.

In [ ]:
# vamos a hacer la prueba con el fichero textos.py que os he mandado antes de comenzar y que debería estar
# en la carpeta mislibrerias
with open('mislibrerias/textos.py', 'r') as f: # la r es de read
    texto = f.read()
    f.close()

In [ ]:
# echo esto, podemos comprobar el contenido de la variable texto
print(texto)

#### Para Guardar.

In [ ]:
# ahora guardaremos nuestro texto en otro fichero
# al escribir, si usamos w, reemplazamos el fichero en caso de existir
with open('mislibrerias/textos2.py', 'w') as f: # la w es de write
    f.write(texto)
    f.write("\n") # añadimos un salto de línea para que lo próximo que escribamos este en la siguiente
    f.close() # importante hacer el close despues de guardar para poder acceder al fichero

Tras ejecutar esto, verás que se ha creado un fichero llamado **textos2.py** en **mislibrerias** con el mismo contenido que **textos.py**.

Elimínalo para evitar problemas a la hora de usar librerías.

Eso es todo por hoy, a por ello!

## Solución

In [ ]:
import requests

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
import re

In [ ]:
from mislibrerias import textos

1. Utiliza requests para pedir la página.

In [ ]:
url = "https://misssushi.es/localiza-tu-misssushi-mas-cercano/"

In [ ]:
req = requests.get(url)

In [ ]:
source = req.text

2. Inspecciona la página para obtener los resultados con BeautifulSoup y genera una lista para cada fila de nuestro excel donde las posiciones sean las siguientes.
- 0: El nombre del restaurante.
- 1: La dirección.
- 2: El teléfono.
- 3: El email.

Ten en cuenta que:
- No es lo mismo el código de una página que lo que vemos, es decir, puede haber información en el código que no estemos viendo...
- En la página que estamos consultado no aparece la información que deseamos, sino el enlace a la información de cada restaurante.
- Quizás te resulte útil hacer el proceso a mano para uno de los restaurantes y crear a partir de ello una función para hacer con todos lo mismo de manera automática.

Información extra:
- Puedes utilizar la función [join()](https://docs.python.org/3/library/stdtypes.html#str.join) de la clase **str** para generar cada registro de nuestro csv.
- Aquí tienes un ejemplo de como buscar links dentro de una página web con  y sin expresiones regulares:

    ```python
    links = [elemento.get('href') for elemento in soup.find_all('a', href=True)] # sin regex, coge todos
    links = [elemento.get('href') for elemento in soup.find_all('a', href=re.compile(r'.*ejemplo'))] # con regex, solo las que lo cumplen
    ```

In [ ]:
soup = BeautifulSoup(source, "html")

In [ ]:
links = [each.get('href') for each in soup.find_all('a', href=re.compile(r'.*restaurantes.*'))]

In [ ]:
def get_contact_info(link):
    source = requests.get(link).text
    soup = BeautifulSoup(source, 'html')
    info = soup.find('div', attrs={'class': 'info-restaurant-item-info'})
    r_name = textos.clean_str(info.find('h1').text)
    r_address = textos.clean_str(info.find('address').text)
    r_phone = textos.clean_str(info.find('div', attrs={'class': 'phone-link'}).text)
    r_mail = textos.clean_str(info.find('a', href=re.compile(r'mailto.*')).text)
    return [r_name, r_address, r_phone, r_mail]    

In [ ]:
results = [get_contact_info(link) for link in links]

In [ ]:
with open('restaurantes_ms.csv', 'w') as f:
    for each in results:
        result_as_string = ';'.join(each)
        f.write(result_as_string)
        f.write("\n")
    f.close()